In [17]:
import pandas as pd
df = pd.read_csv('ohou_urls.csv')
df['제목'] = 0 # 집들이 글 제목
df['작성일자'] = 0
df['평수'] = 0 # 평수 (1-9/10/20/30/40/50/60/70이상)
df['가족형태'] = 0 # 가족형태 (싱글라이프/신혼부부/아기가있는집/취학자녀가있는집/부모님과함꼐사는집/기타)
df['공간'] = 0 # 공간 (원룸&오피스텔/아파트/빌라&연합/단독주택/사무공간/상업공간/기타)
df['스타일'] = 0 # 스타일 (모던/미니멀&심플/내추럴/북유럽/빈티지&레트로/클래식&앤틱/프렌치&프로방스/러블리&로맨틱/인더스트리얼/한국&아시아/유니크&믹스매치)
df['세부공사'] = 0 # 세부공사 (헤링본 마루/대리석 바닥/ 원목마루/포세린타일/폴링싱타일/주방리모델링/조명시공/폴딩도어/중문/가벽&파티션/슬라이딩도어/아트월/발코니확장)
df['작업'] = 0 # 작업 (셀프,DIY/반셀프/전문가)
df['지역'] = 0 # 지역
df['분야'] = 0 # 분야 (리모델링/홈스타일링/부분공사/건축)
df['예산'] = 0 # 예산 (1백미만/1백/2백/3백/4백/5백/1천/2천/3천/4천이상)
df['스크랩'] = 0
df['좋아요'] = 0
df['댓글'] = 0
df['조회'] = 0


In [18]:
df.columns[2:]

Index(['작성일자', '평수', '가족형태', '공간', '스타일', '세부공사', '작업', '지역', '분야', '예산',
       '스크랩', '좋아요', '댓글', '조회'],
      dtype='object')

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as BS
import pandas as pd
from IPython.display import display

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = chrome_options)
# 페이지가 모두 로드될 때까지 3초 기다려주도록 처리
driver.implicitly_wait(3)

for idx, url in enumerate(df['url'][:700]):
    # Selenium을 통해 실제 크롬창에서 접속한 것과 동일하게 해당 URL내용을 가져옴
    driver.get(df['url'][idx])
    # 소스를 읽어 BeautifulSoup를 이용해 파싱
    html = driver.page_source
    soup = BS(html, 'html.parser')
    
    title = soup.select('body > div.page > div > div > main > div.content-detail-content-section > div.content-detail-content-section__content > header > h1')
    date = soup.select('body > div.page > div > div > main > div.content-detail-content-section > div.content-detail-content-section__content > header > div.content-detail-header__bottom > a > div.content-detail-header__author-date')
    texts = soup.select('body > div.page > div > div > main > div.content-detail-content-section > div.content-detail-content-section__content > section > div > div.project-detail-foldable__content > dl > div')
    reactions = soup.select('body > div.page > div > div > main > div.content-detail-footer > dl')
    
    df['제목'][idx] = title[0].text
    df['작성일자'][idx] = date[0].text
    
    text_dic = {}
    for text in texts:
        text_dic[text.find('dt').text] = text.find('dd').text
    for react in reactions[0].find_all('div'):
        text_dic[react.find('dt').text] = react.find('dd').text

    for col in df.columns[2:]: 
        if col in text_dic:
            df[col][idx] = text_dic[col]
            
driver.close()

In [20]:
nae_house = df
nae_house.to_csv('0-700.csv')

In [21]:
nae_house.head()

,url,제목,작성일자,평수,가족형태,공간,스타일,세부공사,작업,지역,분야,예산,스크랩,좋아요,댓글,조회
0,https://ohou.se/projects/119997/detail?affect_...,앱도 집도 사용성이 우선! UI/UX 디자이너의 집,3주 전,30평,신혼부부,아파트,내추럴,0,반셀프,0,리모델링,0,"1,328",412,40,"26,961"
1,https://ohou.se/projects/123100/detail?affect_...,탐나는 포인트가 가득! 매력이 넘치는 숲속의 집,2주 전,31평,신혼부부,아파트,모던,0,전문가,경기도 고양시,리모델링,0,"1,244",360,92,"30,656"
2,https://ohou.se/projects/122245/detail?affect_...,원룸러 주목! 가구 배치로 고통받는 이들을 위한 바이블,3주 전,7평,싱글라이프,원룸&오피스텔,모던,0,셀프•DIY,서울특별시,홈스타일링,0,"1,291",235,42,"28,248"
3,https://ohou.se/projects/121087/detail?affect_...,"점점 내 취향에 가까운 공간으로, 포근포근 빈티지 원룸",3주 전,6평,싱글라이프,원룸&오피스텔,빈티지&레트로,0,셀프•DIY,0,홈스타일링,0,"1,267",215,36,"20,204"
4,https://ohou.se/projects/123059/detail?affect_...,구조와 스타일링이 다채롭게 바뀌는 1.5룸 라이프,2주 전,8평,싱글라이프,원룸&오피스텔,빈티지&레트로,0,셀프•DIY,경상남도 김해시,홈스타일링,0,"1,193",212,63,"24,093"
